In [134]:
import os
from glob import glob
import yaml
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import json
from scipy.stats import wilcoxon
from itertools import combinations
import scienceplots
plt.style.use('science')

In [116]:
def extract_scores(path, key='mean'):
    with open(path, 'r') as f:
        scores = json.load(f)
    all_det_scores = {k: v[key] for (k, v) in filter(lambda x: 'all_det' in x[0], scores.items())}
    bl_det_scores = {k: v[key] for (k, v) in filter(lambda x: 'yer_det' in x[0], scores.items())}
    all_prob_scores = {k: v[key] for (k, v) in filter(lambda x: 'all_pro' in x[0], scores.items())}
    bl_prob_scores = {k: v[key] for (k, v) in filter(lambda x: 'yer_pro' in x[0], scores.items())}
    return all_det_scores, bl_det_scores, all_prob_scores, bl_prob_scores

### Mean

In [100]:
all_det_scores = {}
boundary_layer_det_scores = {}
all_prob_scores = {}
boundary_layer_prob_scores = {}

all_det, bl_det, all_prob, bl_prob = extract_scores('../experiments/data/outputs/svgp/scores.json')

all_det_scores['svgp'] = all_det
boundary_layer_det_scores['svgp'] = bl_det
all_prob_scores['svgp'] = all_prob
boundary_layer_prob_scores['svgp'] = bl_prob

for dirpath in glob('../experiments/data/outputs/ablation/*'):
    path = os.path.join(dirpath, 'scores.json')
    all_det, bl_det, all_prob, bl_prob = extract_scores(path)
    dirname = os.path.basename(dirpath)
    all_det_scores[dirname] = all_det
    boundary_layer_det_scores[dirname] = bl_det
    all_prob_scores[dirname] = all_prob
    boundary_layer_prob_scores[dirname] = bl_prob

In [105]:
det_columns = ['Bias98', 'Corr', 'MAE', 'Bias', 'RMSE']
prob_columns = ['ICI', 'Calib95', 'ELBO']
index = ['SVGP', 'Additive kernel', 'Meteo only', 'GP only', 'Spatiotemporal only', 'Product kernel', 'Joint Matern kernel']

index_order = ['SVGP', 'GP only', 'Spatiotemporal only', 'Meteo only',
               'Product kernel', 'Additive kernel', 'Joint Matern kernel']
det_cols_order = ['RMSE', 'MAE', 'Corr', 'Bias', 'Bias98']
prob_cols_order = ['ELBO', 'Calib95', 'ICI']

all_det_df = pd.DataFrame(all_det_scores).T
all_det_df.index = index
all_det_df.index.name = 'Model'
all_det_df.columns = det_columns
all_det_df = all_det_df.reindex(index_order)
all_det_df = all_det_df[det_cols_order]
all_det_df['Region'] = 'Entire column'

boundary_layer_det_df = pd.DataFrame(boundary_layer_det_scores).T
boundary_layer_det_df.index = index
boundary_layer_det_df.index.name = 'Model'
boundary_layer_det_df.columns = det_columns
boundary_layer_det_df = boundary_layer_det_df.reindex(index_order)
boundary_layer_det_df = boundary_layer_det_df[det_cols_order]
boundary_layer_det_df['Region'] = 'Boundary layer'

all_prob_df = pd.DataFrame(all_prob_scores).T
all_prob_df.index = index
all_prob_df.index.name = 'Model'
all_prob_df.columns = prob_columns
all_prob_df = all_prob_df.reindex(index_order)
all_prob_df = all_prob_df[prob_cols_order]
all_prob_df['Region'] = 'Entire column'

boundary_layer_prob_df = pd.DataFrame(boundary_layer_prob_scores).T
boundary_layer_prob_df.index = index
boundary_layer_prob_df.index.name = 'Model'
boundary_layer_prob_df.columns = prob_columns
boundary_layer_prob_df = boundary_layer_prob_df.reindex(index_order)
boundary_layer_prob_df = boundary_layer_prob_df[prob_cols_order]
boundary_layer_prob_df['Region'] = 'Boundary layer'

mean_deterministic_df = pd.concat([all_det_df, boundary_layer_det_df]).set_index('Region', append=True).swaplevel()
mean_probabilistic_df = pd.concat([all_prob_df, boundary_layer_prob_df]).set_index('Region', append=True).swaplevel()

In [106]:
mean_deterministic_df

RMSE       MAE      Corr  \
Region         Model                                               
Entire column  SVGP                 0.000033  0.000006  0.691812   
               GP only              0.000034  0.000005  0.700376   
               Spatiotemporal only  0.000041  0.000007  0.512172   
               Meteo only           0.000035  0.000006  0.643072   
               Product kernel       0.000036  0.000006  0.621906   
               Additive kernel      0.000034  0.000006  0.650179   
               Joint Matern kernel  0.000034  0.000006  0.670585   
Boundary layer SVGP                 0.000061  0.000017  0.681127   
               GP only              0.000063  0.000015  0.691111   
               Spatiotemporal only  0.000075  0.000017  0.535618   
               Meteo only           0.000064  0.000018  0.625170   
               Product kernel       0.000066  0.000017  0.601654   
               Additive kernel      0.000063  0.000018  0.632228   
               Joint Matern kernel  0.000063  0.000016  0.659058   

                                            Bias    Bias98  
Region         Model                                        
Entire column  SVGP                 3.409000e-07  0.000002  
               GP only             -8.642000e-07 -0.000012  
               Spatiotemporal only -2.412100e-06 -0.000041  
               Meteo only           2.271000e-07  0.000005  
               Product kernel      -5.425000e-07 -0.000006  
               Additive kernel      5.220000e-07  0.000003  
               Joint Matern kernel -1.331000e-07 -0.000007  
Boundary layer SVGP                 1.485100e-06 -0.000038  
               GP only             -3.313600e-06 -0.000062  
               Spatiotemporal only -1.288320e-05 -0.000117  
               Meteo only           1.730000e-06 -0.000033  
               Product kernel      -2.509400e-06 -0.000046  
               Additive kernel      2.233500e-06 -0.000031  
               Joint Matern kernel -1.639100e-06 -0.000048

In [107]:
mean_probabilistic_df

ELBO   Calib95       ICI
Region         Model                                             
Entire column  SVGP                 12.934785  0.892072  0.095790
               GP only              12.905880  0.884873  0.088296
               Spatiotemporal only  13.081722  0.915776  0.071491
               Meteo only           12.702387  0.824581  0.138206
               Product kernel       12.993307  0.910402  0.069905
               Additive kernel      12.937154  0.880631  0.110488
               Joint Matern kernel  12.992290  0.909124  0.096185
Boundary layer SVGP                 10.671634  0.977671  0.062395
               GP only              10.599072  0.969878  0.091848
               Spatiotemporal only  10.406647  0.905476  0.165379
               Meteo only           10.643797  0.969069  0.063491
               Product kernel       10.522968  0.957258  0.101240
               Additive kernel      10.694956  0.974602  0.049671
               Joint Matern kernel  10.657718  0.975880  0.077614

### Stddev

In [108]:
all_det_scores = {}
boundary_layer_det_scores = {}
all_prob_scores = {}
boundary_layer_prob_scores = {}

all_det, bl_det, all_prob, bl_prob = extract_scores('../experiments/data/outputs/svgp/scores.json', key='std')

all_det_scores['svgp'] = all_det
boundary_layer_det_scores['svgp'] = bl_det
all_prob_scores['svgp'] = all_prob
boundary_layer_prob_scores['svgp'] = bl_prob

for dirpath in glob('../experiments/data/outputs/ablation/*'):
    path = os.path.join(dirpath, 'scores.json')
    all_det, bl_det, all_prob, bl_prob = extract_scores(path, key='std')
    dirname = os.path.basename(dirpath)
    all_det_scores[dirname] = all_det
    boundary_layer_det_scores[dirname] = bl_det
    all_prob_scores[dirname] = all_prob
    boundary_layer_prob_scores[dirname] = bl_prob

In [109]:
det_columns = ['Bias98', 'Corr', 'MAE', 'Bias', 'RMSE']
prob_columns = ['ICI', 'Calib95', 'ELBO']
index = ['SVGP', 'Additive kernel', 'Meteo only', 'GP only', 'Spatiotemporal only', 'Product kernel', 'Joint Matern kernel']

index_order = ['SVGP', 'GP only', 'Spatiotemporal only', 'Meteo only',
               'Product kernel', 'Additive kernel', 'Joint Matern kernel']
det_cols_order = ['RMSE', 'MAE', 'Corr', 'Bias', 'Bias98']
prob_cols_order = ['ELBO', 'Calib95', 'ICI']

all_det_df = pd.DataFrame(all_det_scores).T
all_det_df.index = index
all_det_df.index.name = 'Model'
all_det_df.columns = det_columns
all_det_df = all_det_df.reindex(index_order)
all_det_df = all_det_df[det_cols_order]
all_det_df['Region'] = 'Entire column'

boundary_layer_det_df = pd.DataFrame(boundary_layer_det_scores).T
boundary_layer_det_df.index = index
boundary_layer_det_df.index.name = 'Model'
boundary_layer_det_df.columns = det_columns
boundary_layer_det_df = boundary_layer_det_df.reindex(index_order)
boundary_layer_det_df = boundary_layer_det_df[det_cols_order]
boundary_layer_det_df['Region'] = 'Boundary layer'

all_prob_df = pd.DataFrame(all_prob_scores).T
all_prob_df.index = index
all_prob_df.index.name = 'Model'
all_prob_df.columns = prob_columns
all_prob_df = all_prob_df.reindex(index_order)
all_prob_df = all_prob_df[prob_cols_order]
all_prob_df['Region'] = 'Entire column'

boundary_layer_prob_df = pd.DataFrame(boundary_layer_prob_scores).T
boundary_layer_prob_df.index = index
boundary_layer_prob_df.index.name = 'Model'
boundary_layer_prob_df.columns = prob_columns
boundary_layer_prob_df = boundary_layer_prob_df.reindex(index_order)
boundary_layer_prob_df = boundary_layer_prob_df[prob_cols_order]
boundary_layer_prob_df['Region'] = 'Boundary layer'

stddev_deterministic_df = pd.concat([all_det_df, boundary_layer_det_df]).set_index('Region', append=True).swaplevel()
stddev_probabilistic_df = pd.concat([all_prob_df, boundary_layer_prob_df]).set_index('Region', append=True).swaplevel()

In [110]:
mean_deterministic_df

RMSE       MAE      Corr  \
Region         Model                                               
Entire column  SVGP                 0.000033  0.000006  0.691812   
               GP only              0.000034  0.000005  0.700376   
               Spatiotemporal only  0.000041  0.000007  0.512172   
               Meteo only           0.000035  0.000006  0.643072   
               Product kernel       0.000036  0.000006  0.621906   
               Additive kernel      0.000034  0.000006  0.650179   
               Joint Matern kernel  0.000034  0.000006  0.670585   
Boundary layer SVGP                 0.000061  0.000017  0.681127   
               GP only              0.000063  0.000015  0.691111   
               Spatiotemporal only  0.000075  0.000017  0.535618   
               Meteo only           0.000064  0.000018  0.625170   
               Product kernel       0.000066  0.000017  0.601654   
               Additive kernel      0.000063  0.000018  0.632228   
               Joint Matern kernel  0.000063  0.000016  0.659058   

                                            Bias    Bias98  
Region         Model                                        
Entire column  SVGP                 3.409000e-07  0.000002  
               GP only             -8.642000e-07 -0.000012  
               Spatiotemporal only -2.412100e-06 -0.000041  
               Meteo only           2.271000e-07  0.000005  
               Product kernel      -5.425000e-07 -0.000006  
               Additive kernel      5.220000e-07  0.000003  
               Joint Matern kernel -1.331000e-07 -0.000007  
Boundary layer SVGP                 1.485100e-06 -0.000038  
               GP only             -3.313600e-06 -0.000062  
               Spatiotemporal only -1.288320e-05 -0.000117  
               Meteo only           1.730000e-06 -0.000033  
               Product kernel      -2.509400e-06 -0.000046  
               Additive kernel      2.233500e-06 -0.000031  
               Joint Matern kernel -1.639100e-06 -0.000048

In [111]:
mean_probabilistic_df

ELBO   Calib95       ICI
Region         Model                                             
Entire column  SVGP                 12.934785  0.892072  0.095790
               GP only              12.905880  0.884873  0.088296
               Spatiotemporal only  13.081722  0.915776  0.071491
               Meteo only           12.702387  0.824581  0.138206
               Product kernel       12.993307  0.910402  0.069905
               Additive kernel      12.937154  0.880631  0.110488
               Joint Matern kernel  12.992290  0.909124  0.096185
Boundary layer SVGP                 10.671634  0.977671  0.062395
               GP only              10.599072  0.969878  0.091848
               Spatiotemporal only  10.406647  0.905476  0.165379
               Meteo only           10.643797  0.969069  0.063491
               Product kernel       10.522968  0.957258  0.101240
               Additive kernel      10.694956  0.974602  0.049671
               Joint Matern kernel  10.657718  0.975880  0.077614

## Statistical significance

In [144]:
def paired_wilcoxon(df):
    rows = list(df.index)
    cols = df.columns
    W_dict = {}
    p_dict = {}
    for k, metric in enumerate(rows):
        W_values = np.zeros((len(cols), len(cols)))
        p_values = np.zeros((len(cols), len(cols)))
        metric_df = df.loc[metric]
        for i, j in combinations(range(len(cols)), 2):
            W, p = wilcoxon(x=metric_df[cols[i]],
                            y=metric_df[cols[j]])
            W_values[i, j] = W
            p_values[i, j] = p
        W_values =  W_values + W_values.T - np.eye(len(cols))
        p_values = p_values + p_values.T - np.eye(len(cols))
        W_dict[metric] = pd.DataFrame(data=W_values, columns=cols, index=cols)
        p_dict[metric] = pd.DataFrame(data=p_values, columns=cols, index=cols)
    return W_dict, p_dict

In [ ]:
all_det_scores = {}
boundary_layer_det_scores = {}
all_prob_scores = {}
boundary_layer_prob_scores = {}

all_det, bl_det, all_prob, bl_prob = extract_scores('../experiments/data/outputs/svgp/scores.json', key='values')

all_det_scores['svgp'] = all_det
boundary_layer_det_scores['svgp'] = bl_det
all_prob_scores['svgp'] = all_prob
boundary_layer_prob_scores['svgp'] = bl_prob

for dirpath in glob('../experiments/data/outputs/ablation/*'):
    path = os.path.join(dirpath, 'scores.json')
    all_det, bl_det, all_prob, bl_prob = extract_scores(path, key='values')
    dirname = os.path.basename(dirpath)
    all_det_scores[dirname] = all_det
    boundary_layer_det_scores[dirname] = bl_det
    all_prob_scores[dirname] = all_prob
    boundary_layer_prob_scores[dirname] = bl_prob

In [ ]:
det_columns = ['Bias98', 'Corr', 'MAE', 'Bias', 'RMSE']
prob_columns = ['ICI', 'Calib95', 'ELBO']
index = ['SVGP', 'Additive kernel', 'Meteo only', 'GP only', 'Spatiotemporal only', 'Product kernel', 'Joint Matern kernel']

index_order = ['SVGP', 'GP only', 'Spatiotemporal only', 'Meteo only',
               'Product kernel', 'Additive kernel', 'Joint Matern kernel']
det_cols_order = ['RMSE', 'MAE', 'Corr', 'Bias', 'Bias98']
prob_cols_order = ['ELBO', 'Calib95', 'ICI']


In [ ]:
W_dict, p_dict = paired_wilcoxon(df)